In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/medmnist-chest/chestmnist_compositionalauc_512bs_0.05_SLR_RLRP_7275.pth


In [2]:
%%capture
!pip install libauc==1.2.0
!pip install medmnist
!pip install torchio

In [3]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

from libauc.models import resnet18
from libauc.sampler import DualSampler
from libauc.metrics import auc_prc_score

import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image, ImageFilter
from torchio import Image

from libauc.losses import AUCMLoss, CrossEntropyLoss, CompositionalAUCLoss
from libauc.optimizers import PESG, Adam
# from libauc.models import densenet121 as DenseNet121


from PIL import Image
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torchvision.models as models
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

In [4]:
from torchio.transforms import RandomAffine, RandomFlip, RandomNoise, RandomGamma
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=28, crop_size=24, mode='train', kernel_size=3):
        self.images = images.astype(np.uint8)
        self.targets = targets
        self.mode = mode

        self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                              ])
        
        
        # for loss function
        self.pos_indices = np.flatnonzero(targets==1)
        self.pos_index_map = {}
        for i, idx in enumerate(self.pos_indices):
            self.pos_index_map[idx] = i

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        #image = torch.from_numpy(image).unsqueeze(0) 
        if self.mode == 'train':
           idx = self.pos_index_map[idx] if idx in self.pos_indices else -1
           #image = image.filter(ImageFilter.GaussianBlur(radius=3))
           #image = image.filter(ImageFilter.BLUR)
           #image = image.filter(ImageFilter.UnsharpMask(radius=3, percent=150, threshold=3))

           image = self.transform_train(image)
        else:
           #image = image.filter(ImageFilter.GaussianBlur(radius=3))
           #image = image.filter(ImageFilter.BLUR)
           #image = image.filter(ImageFilter.UnsharpMask(radius=3, percent=150, threshold=3))
           image = self.transform_test(image)
        return idx, image, target 

In [5]:
device = torch.device(0 if torch.cuda.is_available() else 'cpu')

### ChestMNIST

In [6]:
data_flag = 'chestmnist'

download = True

info = INFO[data_flag]

DataClass = getattr(medmnist, info['python_class'])
test_dataset = DataClass(split='test', download=download)
test_images = np.array([np.asarray(image) for (image, target) in test_dataset])
test_labels = [target for (image, target) in test_dataset]
batch_size = 512
testSet = ImageDataset(test_images, test_labels, mode='test')
test_loader = torch.utils.data.DataLoader(testSet, batch_size=batch_size, shuffle=True, num_workers=2)


100%|██████████| 82802576/82802576 [02:34<00:00, 536417.65it/s]


In [7]:
print(f'Testing ...')  
PATH = '/kaggle/input/medmnist-chest/chestmnist_compositionalauc_512bs_0.05_SLR_RLRP_7275.pth' 
model = torch.load(PATH)
best_val_auc=0.0
with torch.no_grad():    
     test_pred = []
     test_true = [] 
     for jdx, (index, data, targets) in enumerate(test_loader):
         test_data, test_labels = data.to(device), targets.to(device)
         test_data=test_data.repeat(1,3,1,1)
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_labels.cpu().detach().numpy())
 
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     val_auc_mean =  roc_auc_score((test_true), test_pred) 
     print('Test_AUC=%.7f'%( val_auc_mean))

Testing ...
Test_AUC=0.7274890
